In [2]:
import numpy as np
shape = 4096000
a = np.arange(shape, dtype=np.int64)
b = np.empty_like(a, dtype=np.int64)
print(a)
# prints [0 1 2 ... 4095997 4095998 4095999]

[      0       1       2 ... 4095997 4095998 4095999]


In [2]:
def process_fn():
    # iterating and squaring each element in a and store to b
    with np.nditer([a, b],
                   op_flags=[['readonly'], ['readwrite']]) as it:
        with it:
           for x,y in it:
                y[...] = x**2
%timeit process_fn()  # 3.55 s ± 22.7 ms per loop
print(b)
# prints [0 1 4 ... 16777191424009 16777199616004 16777207808001]

4.09 s ± 69.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[             0              1              4 ... 16777191424009
 16777199616004 16777207808001]


In [4]:
import daisy
from funlib.persistence import Array
from funlib.geometry import Roi, Coordinate
import zarr
shape = 4096000
block_shape = 1024*16
# input array is wrapped in `Array` for easy of `Roi` indexing
a = Array(np.arange(shape, dtype=np.int64),
                roi=Roi((0,), shape),
                voxel_size=(1,))
# to parallelize across processes, we need persistent read/write arrays
# we'll use zarr here to do do that
b = zarr.open_array(zarr.TempStore(), 'w', (shape,),
                    chunks=(block_shape,),
                    dtype=np.int64)
# output array is wrapped in Array for easy of Roi indexing
b = Array(b,
                roi=Roi((0,), shape),
                voxel_size=(1,))

/home/ubuntu/miniconda3/envs/lsd_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# same process function as previously, but with additional code
# to read and write data to persistent arrays
def process_fn_daisy(block):
    a_sub = a[block.read_roi].to_ndarray()
    b_sub = np.empty_like(a_sub)
    with np.nditer([a_sub, b_sub],
                   op_flags=[['readonly'], ['readwrite']],
                  ) as it:
        with it:
           for x,y in it:
                y[...] = x**2
    
    b[block.write_roi] = b_sub


total_roi = Roi((0,), shape)  # total ROI to map process over
block_roi = Roi((0,), (block_shape,))  # block ROI for parallel processing
# creating a Daisy task, note that we do not specify how each
# worker should read/write to input/output arrays
task = daisy.Task(
    total_roi=total_roi,
    read_roi=block_roi,
    write_roi=block_roi,
    process_function=process_fn_daisy,
    num_workers=1,
    task_id='square',
)
daisy.run_blockwise([task])

# %timeit daisy.run_blockwise([task])  # 1.26 s ± 16.1 ms per loop
print(b.to_ndarray())
# prints [0 1 4 ... 16777191424009 16777199616004 16777207808001]

square ✔: 100%|██████████| 250/250 [00:15<00:00, 16.44blocks/s, ⧗=0, ▶=0, ✔=250, ✗=0, ∅=0]



Execution Summary
-----------------

  Task square:

    num blocks : 250
    completed ✔: 250 (skipped 0)
    failed    ✗: 0
    orphaned  ∅: 0

    all blocks processed successfully
[             0              1              4 ... 16777191424009
 16777199616004 16777207808001]


In [1]:
import funlib.persistence

In [2]:
import os 
base_dir = '/data/base'
experiment = '3M-APP-SCN'
setup = 'setup04'
experiment_dir = os.path.join(base_dir, experiment)
print(experiment_dir)
train_dir = os.path.join(experiment_dir, '02_train')
print(train_dir)

network_dir = os.path.join(experiment, setup, '400000')
print(network_dir)

/data/base/3M-APP-SCN
/data/base/3M-APP-SCN/02_train
3M-APP-SCN/setup04/400000
